In [ ]:
# List bond angles

from src.local_features import NMR_local

structure = structure_tensors_filtered[17]["structure"]
first_coord_dict = NMR_local(structure).first_neighbours
first_coord_dict

import itertools
from pymatgen.util.coord import get_angle

center_index = 2
first_neighbours = first_coord_dict[center_index]
site_combos = itertools.combinations(first_neighbours, 2)
angles_differences = []
for combo in site_combos:
    v1 = combo[0]["site"].coords - structure[int(center_index)].coords
    v2 = combo[1]["site"].coords - structure[int(center_index)].coords
    angles_differences.append(get_angle(v1, v2, units="degrees"))
angles_differences.sort(reverse=True)
angles_differences

In [ ]:
# chemical environment of a site

from pymatgen.analysis.chemenv.coordination_environments.coordination_geometry_finder import (
    LocalGeometryFinder,
)
from matminer.featurizers.site.fingerprint import ChemEnvSiteFingerprint
from pymatgen.analysis.chemenv.coordination_environments.chemenv_strategies import (
    MultiWeightsChemenvStrategy,
)

cetypes = [
    "TL:3",
    "TY:3",
    "TS:3",
    "T:4",
    "S:4",
    "SY:4",
    "SS:4",
    "PP:5",
    "S:5",
    "T:5",
    "O:6",
    "T:6",
    "PP:6",
]
lgf = LocalGeometryFinder()
lgf.setup_parameters(
    centering_type="centroid",
    include_central_site_in_centroid=True,
    structure_refinement=lgf.STRUCTURE_REFINEMENT_NONE,
)
envfingerprint = ChemEnvSiteFingerprint(
    cetypes,
    MultiWeightsChemenvStrategy.stats_article_weights_parameters(),
    lgf,
)
labels = envfingerprint.feature_labels()
dict(
    zip(
        labels, envfingerprint.featurize(structure_tensors_filtered[15]["structure"], 4)
    )
)